In [ ]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import seaborn as sns
from skimage.transform import resize
from skimage.measure import block_reduce
import tensorflow as tf
#from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy import where
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
#In this competition tensorflow is not updated, then we use keras to load model and weights
from keras.models import model_from_json, load_model
import json
from sklearn.model_selection import KFold

In [ ]:
import pydicom
import os
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import mean_absolute_error

In [ ]:
ROOT = "../input/osic-pulmonary-fibrosis-progression"
#DESIRED_SIZE = 256 # Memory issue
DESIRED_SIZE = 256

In [ ]:
tr = pd.read_csv(f"{ROOT}/train.csv")
tr.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])
chunk = pd.read_csv(f"{ROOT}/test.csv")

print("add infos")
sub = pd.read_csv(f"{ROOT}/sample_submission.csv")
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']]
sub = sub.merge(chunk.drop('Weeks', axis=1), on="Patient")

In [ ]:
tr['WHERE'] = 'train'
chunk['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = tr.append([chunk, sub])

In [ ]:
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

In [ ]:
base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)

In [ ]:
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base

In [ ]:
COLS = ['Sex','SmokingStatus']
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)
#=================

In [ ]:
#
data['age'] = (data['Age'] - data['Age'].min() ) / ( data['Age'].max() - data['Age'].min() )
data['BASE'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
data['percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )
FE += ['age','percent','week','BASE']

In [ ]:
tr = data.loc[data.WHERE=='train']
chunk = data.loc[data.WHERE=='val']
sub = data.loc[data.WHERE=='test']
del data

In [ ]:
tr.shape, chunk.shape, sub.shape

### Quick Image processing

In [ ]:
def get_images(df, how="train"):
    xo = []
    p = []
    w  = []
    for i in tqdm(range(df.shape[0])):
        patient = df.iloc[i,0]
        week = df.iloc[i,1]
        try:
            img_path = f"{ROOT}/{how}/{patient}/{week}.dcm"
            ds = pydicom.dcmread(img_path)
            im = Image.fromarray(ds.pixel_array)
            im = im.resize((DESIRED_SIZE,DESIRED_SIZE)) 
            im = np.array(im)
            xo.append(im[np.newaxis,:,:])
            p.append(patient)
            w.append(week)
        except:
            pass
    data = pd.DataFrame({"Patient":p,"Weeks":w})
    return np.concatenate(xo, axis=0), data

In [ ]:
x, df_tr = get_images(tr, how="train")

In [ ]:
x.shape, df_tr.shape

In [ ]:
idx = np.random.randint(x.shape[0])
plt.imshow(x[idx], cmap=plt.cm.bone)
plt.show()

In [ ]:
df_tr = df_tr.merge(tr, how="left", on=['Patient', 'Weeks'])

In [ ]:
y = df_tr['FVC'].values
z = df_tr[FE].values

In [ ]:
y

In [ ]:
z.shape

### BASELINE CNN 

In [ ]:
# import tensorflow.keras.backend as K
# import tensorflow.keras.layers as L
# import tensorflow.keras.models as M

In [ ]:
C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
#=============================#
def kloss(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 1]
    fvc_pred = y_pred[:, 0]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)
#=============================#
def kmae(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    spread = tf.abs( (y_true[:, 0] -  y_pred[:, 0])  / y_true[:, 0] )
    #spred = tf.square(y_true, y_pred[:, 0])
    return K.mean(spread)
#=============================#

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * kloss(y_true, y_pred) + (1 - _lambda)*kmae(y_true, y_pred)
    return loss
#=================

# def make_model():
#     inp = L.Input((DESIRED_SIZE,DESIRED_SIZE), name="input")
#     z = L.Input((9,), name="Patient")
#     x = L.Conv1D(50, 4, activation="relu", name="conv1")(inp)
#     x = L.MaxPool1D(2, name='pool1')(x)
    
#     #x = L.Dropout(0.2)(x)
#     x = L.Conv1D(50, 4, activation="relu", name="conv2")(x)
#     x = L.MaxPool1D(2, name='pool2')(x)
    
#     #x = L.Dropout(0.2)(x)
#     x = L.Conv1D(50, 4, activation="relu", name="conv3")(x)
#     x = L.MaxPool1D(2, name='pool3')(x)
    
#     x = L.Flatten(name="features")(x)
#     x = L.Dense(50, activation="relu", name="d1")(x)
#     l = L.Dense(10, activation="relu", name="d2")(z)
#     x = L.Concatenate(name="combine")([x, l])
#     x = L.Dense(50, activation="relu", name="d3")(x)
#     preds = L.Dense(2, activation="relu", name="preds")(x)
    
#     model = M.Model([inp, z], preds, name="CNN")
#     model.compile(loss=mloss(0.5), optimizer="adam", metrics=[kloss])
#     #model.compile(loss=kmae, optimizer="adam", metrics=[kloss])
#     #model.compile(loss=kloss, optimizer="adam", metrics=[kmae])#
#     return model

### Modified VGG19 use like baseline model

In [ ]:
def VGG19(x):
    
    #Block 1
    x = Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1', kernel_initializer='he_normal')(x)
    x = Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2', kernel_initializer='he_normal')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1', kernel_initializer='he_normal')(x)
    x = Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2', kernel_initializer='he_normal')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#     # Block 3
#     x = Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block3_conv1', kernel_initializer='he_normal')(x)
#     x = Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block3_conv2', kernel_initializer='he_normal')(x)
#     x = Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block3_conv3', kernel_initializer='he_normal')(x)
#     x = Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block3_conv4', kernel_initializer='he_normal')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#     # Block 4
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv1', kernel_initializer='he_normal')(x)
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv2', kernel_initializer='he_normal')(x)
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv3', kernel_initializer='he_normal')(x)
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv4', kernel_initializer='he_normal')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#     # Block 5
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv1', kernel_initializer='he_normal')(x)
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv2', kernel_initializer='he_normal')(x)
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv3', kernel_initializer='he_normal')(x)
#     x = Conv2D(512, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv4', kernel_initializer='he_normal')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    return x

In [ ]:
# Global Constants
LRN2D_NORM=True
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'
USE_BN=True
DROPOUT=0.25
#weight_decay=0.01 #None
weight_decay=None

def conv2D_lrn2d(x,filters,kernel_size,strides=(1,1),padding='same',dilation_rate=(1,1),activation='relu',
                 use_bias=True,kernel_initializer='he_normal',bias_initializer='zeros',
                 kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,
                 kernel_constraint=None,bias_constraint=None,lrn2d_norm=LRN2D_NORM,weight_decay=weight_decay):
    #l2 normalization
    if weight_decay:
        kernel_regularizer=tf.keras.regularizers.l1(weight_decay)
        bias_regularizer=tf.keras.regularizers.l1(weight_decay)
    else:
        kernel_regularizer=None
        bias_regularizer=None
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding,dilation_rate=dilation_rate,
             activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
             bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
             activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    if lrn2d_norm:
        #batch normalization
        x=BatchNormalization()(x)

    return x

#Defining InceptionV1 module
def inception_module(x,params,concat_axis,padding='same',dilation_rate=(1,1),activation='relu',
                     use_bias=True,kernel_initializer='he_normal',bias_initializer='zeros',
                     kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,
                     bias_constraint=None,lrn2d_norm=LRN2D_NORM,weight_decay=weight_decay):
    (branch1,branch2,branch3,branch4)=params
    if weight_decay:
        kernel_regularizer=tf.keras.regularizers.l1(weight_decay)
        bias_regularizer=tf.keras.regularizers.l1(weight_decay)
    else:
        kernel_regularizer=None
        bias_regularizer=None
    #1x1
    pathway1=Conv2D(filters=branch1[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    #1x1->3x3
    pathway2=Conv2D(filters=branch2[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    pathway2=Conv2D(filters=branch2[1],kernel_size=(3,3),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(pathway2)
    #1x1->5x5
    pathway3=Conv2D(filters=branch3[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    pathway3=Conv2D(filters=branch3[1],kernel_size=(5,5),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(pathway3)
    #3x3->1x1
    pathway4=MaxPooling2D(pool_size=(3,3),strides=1,padding=padding,data_format=DATA_FORMAT)(x)
    pathway4=Conv2D(filters=branch4[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(pathway4)

    return concatenate([pathway1,pathway2,pathway3,pathway4],axis=concat_axis)

In [ ]:
#Defining conv2d_bn to inceptionv3_module
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), name=None):
    """Utility function to apply conv + BN.
    Arguments:
    x: input tensor.
    filters: filters in `Conv2D`.
    num_row: height of the convolution kernel.
    num_col: width of the convolution kernel.
    padding: padding mode in `Conv2D`.
    strides: strides in `Conv2D`.
    name: name of the ops; will become `name + '_conv'`
      for the convolution and `name + '_bn'` for the
      batch norm layer.
      Returns:
      Output tensor after applying `Conv2D` and `BatchNormalization`.
      """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    
    x = Conv2D(filters, (num_row, num_col),strides=strides,padding=padding,use_bias=False,name=conv_name)(x)
    x = BatchNormalization(scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    
    return x

In [ ]:
channel_axis=3
def inceptionV3_module(x):
    x = conv2d_bn(x, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    x = conv2d_bn(x, 80, 1, 1, padding='valid')
    x = conv2d_bn(x, 192, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    # mixed 0: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)
    
    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)
    
    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    
    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                           axis=channel_axis,
                           name='mixed0')
    # mixed 1: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)
    
    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)
    
    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    
    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                           axis=channel_axis,
                           name='mixed1')
    
    # mixed 2: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)
    
    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)
    
    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    
    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                           axis=channel_axis,
                           name='mixed2')
    
    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')
    
    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')
    
    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch3x3dbl, branch_pool],
                           axis=channel_axis,
                           name='mixed3')
    
    # mixed 4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)
    
    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)
    
    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    
    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                           axis=channel_axis,
                           name='mixed4')
    
    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1)
        
        branch7x7 = conv2d_bn(x, 160, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)
        
        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
        
        branch_pool = AveragePooling2D((3, 3),
                                              strides=(1, 1),
                                              padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                               axis=channel_axis,
                               name='mixed' + str(5 + i))
        
    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)
    
    branch7x7 = conv2d_bn(x, 192, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)
    
    branch7x7dbl = conv2d_bn(x, 192, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    
    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                           axis=channel_axis,
                           name='mixed7')
    
    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2, 2), padding='valid')
    
    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(
        branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')
    
    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch7x7x3, branch_pool],
                           axis=channel_axis,
                           name='mixed8')
    
    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)
        
        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = concatenate([branch3x3_1, branch3x3_2],
                                       axis=channel_axis,
                                       name='mixed9_' + str(i))
        
        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2],
                                          axis=channel_axis)
        
        branch_pool = AveragePooling2D((3, 3),
                                              strides=(1, 1),
                                              padding='same')(x)
        
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                               axis=channel_axis,
                               name='mixed' + str(9 + i))
        
        return x

In [ ]:
CONCAT_AXIS=3

#Model InceptionV1

def InceptionV1():
    inp = Input((DESIRED_SIZE,DESIRED_SIZE,1), name="input")
    x = inception_module(inp, params=[(128,), (128, 192), (32, 96), (64,)], concat_axis=CONCAT_AXIS)  # 3b
    x = Flatten()(x)
    x = Dropout(DROPOUT)(x)
    preds = Dense(2, activation="relu", name="preds")(x)
    
    #model = tf.keras.Model([inp, z], preds, name="CNN")
    model = tf.keras.Model(inp, preds, name="CNN")
    #opt = tf.keras.optimizers.SGD(learning_rate=0.001)
    #opt = tfa.optimizers.LAMB(learning_rate=0.001)
    #opt = tfa.optimizers.SGDW(learning_rate=0.001,weight_decay=0.001, momentum=0.01) #weight_decay=0.001
    #opt = tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001), 100, 10)
    #opt=tfa.optimizers.AveragedOptimizerWrapper(tf.keras.optimizers.Adam(lr=0.001))
    #loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
    opt=tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001))
    #model.compile(loss=mloss(0.5), optimizer="adam", metrics=[kloss])
    model.compile(loss=mloss(0.5), optimizer=opt, metrics=[kloss])
    #model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    return model

In [ ]:
CONCAT_AXIS=3

#Model GoogleNet_InceptionV1

def GoogleNet():
    inp = Input((DESIRED_SIZE,DESIRED_SIZE,1), name="input")
    z = Input((9,), name="Patient")
    #x = VGG19(inp)
    x = conv2D_lrn2d(inp, 64, (7, 7), 2, padding='same', lrn2d_norm=False)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)
    x = BatchNormalization()(x)

    x = conv2D_lrn2d(x, 64, (1, 1), 1, padding='same', lrn2d_norm=False)

    x = conv2D_lrn2d(x, 192, (3, 3), 1, padding='same', lrn2d_norm=True)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)

    x = inception_module(x, params=[(64,), (96, 128), (16, 32), (32,)], concat_axis=CONCAT_AXIS)  # 3a
    x = inception_module(x, params=[(128,), (128, 192), (32, 96), (64,)], concat_axis=CONCAT_AXIS)  # 3b
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)
    
    x = inception_module(x, params=[(192,), (96, 208), (16, 48), (64,)], concat_axis=CONCAT_AXIS)  # 4a
    x = inception_module(x, params=[(160,), (112, 224), (24, 64), (64,)], concat_axis=CONCAT_AXIS)  # 4b
    x = inception_module(x, params=[(128,), (128, 256), (24, 64), (64,)], concat_axis=CONCAT_AXIS)  # 4c
    x = inception_module(x, params=[(112,), (144, 288), (32, 64), (64,)], concat_axis=CONCAT_AXIS)  # 4d
    x = inception_module(x, params=[(256,), (160, 320), (32, 128), (128,)], concat_axis=CONCAT_AXIS)  # 4e
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)

    x = inception_module(x, params=[(256,), (160, 320), (32, 128), (128,)], concat_axis=CONCAT_AXIS)  # 5a
    x = inception_module(x, params=[(384,), (192, 384), (48, 128), (128,)], concat_axis=CONCAT_AXIS)  # 5b
    x = AveragePooling2D(pool_size=(1, 1), strides=1, padding='valid')(x)

    x = Flatten()(x)
    x = Dropout(DROPOUT)(x)
    preds = Dense(2, activation="relu", name="preds")(x)
    
    #model = tf.keras.Model([inp, z], preds, name="CNN")
    model = tf.keras.Model(inp, preds, name="CNN")
    #opt = tf.keras.optimizers.SGD(learning_rate=0.001)
    #opt = tfa.optimizers.LAMB(learning_rate=0.001)
    #opt = tfa.optimizers.SGDW(learning_rate=0.001,weight_decay=0.001, momentum=0.01) #weight_decay=0.001
    #opt = tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001), 100, 10)
    #opt=tfa.optimizers.AveragedOptimizerWrapper(tf.keras.optimizers.Adam(lr=0.001))
    #loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
    opt=tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001))
    #model.compile(loss=mloss(0.5), optimizer="adam", metrics=[kloss])
    model.compile(loss=mloss(0.5), optimizer=opt, metrics=[kloss])
    #model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    return model

In [ ]:
CONCAT_AXIS=3

#Model InceptionV3

def InceptionV3():
    inp = Input((DESIRED_SIZE,DESIRED_SIZE,1), name="input")
    x = inceptionV3_module(inp)  # 3b
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    preds = Dense(2, activation="relu", name="preds")(x)
    
    #model = tf.keras.Model([inp, z], preds, name="CNN")
    model = tf.keras.Model(inp, preds, name="CNN")
    #opt = tf.keras.optimizers.SGD(learning_rate=0.001)
    #opt = tfa.optimizers.LAMB(learning_rate=0.001)
    #opt = tfa.optimizers.SGDW(learning_rate=0.001,weight_decay=0.001, momentum=0.01) #weight_decay=0.001
    #opt = tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001), 100, 10)
    #opt=tfa.optimizers.AveragedOptimizerWrapper(tf.keras.optimizers.Adam(lr=0.001))
    #loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
    opt=tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001))
    #model.compile(loss=mloss(0.5), optimizer="adam", metrics=[kloss])
    model.compile(loss=mloss(0.5), optimizer=opt, metrics=[kloss])
    #model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    return model

In [ ]:
CONCAT_AXIS=3

#Model GoogleNet_InceptionV1

def InceptionV3_GoogleNet():
    inp = Input((DESIRED_SIZE,DESIRED_SIZE,1), name="input")
    #x = VGG19(inp)
    x = inceptionV3_module(inp)
    x = conv2D_lrn2d(x, 64, (7, 7), 2, padding='same', lrn2d_norm=False)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)
    x = BatchNormalization()(x)

    x = conv2D_lrn2d(x, 64, (1, 1), 1, padding='same', lrn2d_norm=False)

    x = conv2D_lrn2d(x, 192, (3, 3), 1, padding='same', lrn2d_norm=True)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)

    x = inception_module(x, params=[(64,), (96, 128), (16, 32), (32,)], concat_axis=CONCAT_AXIS)  # 3a
    x = inception_module(x, params=[(128,), (128, 192), (32, 96), (64,)], concat_axis=CONCAT_AXIS)  # 3b
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)
    
    x = inception_module(x, params=[(192,), (96, 208), (16, 48), (64,)], concat_axis=CONCAT_AXIS)  # 4a
    x = inception_module(x, params=[(160,), (112, 224), (24, 64), (64,)], concat_axis=CONCAT_AXIS)  # 4b
    x = inception_module(x, params=[(128,), (128, 256), (24, 64), (64,)], concat_axis=CONCAT_AXIS)  # 4c
    x = inception_module(x, params=[(112,), (144, 288), (32, 64), (64,)], concat_axis=CONCAT_AXIS)  # 4d
    x = inception_module(x, params=[(256,), (160, 320), (32, 128), (128,)], concat_axis=CONCAT_AXIS)  # 4e
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)

    x = inception_module(x, params=[(256,), (160, 320), (32, 128), (128,)], concat_axis=CONCAT_AXIS)  # 5a
    x = inception_module(x, params=[(384,), (192, 384), (48, 128), (128,)], concat_axis=CONCAT_AXIS)  # 5b
    x = AveragePooling2D(pool_size=(1, 1), strides=1, padding='valid')(x)

    x = Flatten()(x)
    x = Dropout(DROPOUT)(x)
    preds = Dense(2, activation="relu", name="preds")(x)
    
    #model = tf.keras.Model([inp, z], preds, name="CNN")
    model = tf.keras.Model(inp, preds, name="InceptionV3_GoogleNet")
    
    global_step = tf.Variable(0, trainable=False)
    initial_learning_rate = 0.01
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=1000,#100000
        decay_rate=0.96,
        staircase=True)
    
    #opt = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=lr_schedule)
    #opt = tfa.optimizers.SGDW(learning_rate=0.001,weight_decay=0.001, momentum=0.01) #weight_decay=0.001
    #opt = tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001), 100, 10)
    #loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
    #opt=tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001))
    #model.compile(loss=mloss(0.5), optimizer="adam", metrics=[kloss])
    model.compile(loss=mloss(0.65), optimizer=opt, metrics=[kloss])
    #model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    return model

In [ ]:
#Modified EfficientNet

"""
# Reference
- [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks]
   (https://arxiv.org/abs/1905.11946)
"""

def get_top(x_input):
    """Block top operations
    This functions apply Batch Normalization and Leaky ReLU activation to the input.
    # Arguments:
        x_input: Tensor, input to apply BN and activation  to.
    # Returns:
        Output tensor
    """
    
    x = BatchNormalization()(x_input)
    x = LeakyReLU()(x)
    return x

def get_block(x_input, input_channels, output_channels):
    """MBConv block
    This function defines a mobile Inverted Residual Bottleneck block with BN and Leaky ReLU
    # Arguments
        x_input: Tensor, input tensor of conv layer.
        input_channels: Integer, the dimentionality of the input space.
        output_channels: Integer, the dimensionality of the output space.
            
    # Returns
        Output tensor.
    """

    x = Conv2D(input_channels, kernel_size=(1, 1), padding='same', use_bias=False)(x_input)
    x = get_top(x)
    x = DepthwiseConv2D(kernel_size=(1, 3), padding='same', use_bias=False)(x)
    x = get_top(x)
    x = MaxPooling2D(pool_size=(2, 1), strides=(2, 1))(x)
    x = DepthwiseConv2D(kernel_size=(3, 1), padding='same', use_bias=False)(x)
    x = get_top(x)
    x = Conv2D(output_channels, kernel_size=(2, 1), strides=(1, 2), padding='same', use_bias=False)(x)
    return x


def EffNet(num_classes=2):
    """EffNet
    This function defines a EfficientNet architecture.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        num_classes: Integer, number of classes.
        plot_model: Boolean, whether to plot model architecture or not
    # Returns
        EfficientNet model.
    """
    
    inp = Input((DESIRED_SIZE,DESIRED_SIZE,1), name="input")
    x = get_block(inp, 32, 64)
    x = get_block(x, 64, 128)
    x = get_block(x, 128, 256)
    x = Flatten()(x)
    preds = Dense(num_classes, activation='relu')(x)
    
    #model = tf.keras.Model([inp, z], preds, name="CNN")
    model = tf.keras.Model(inp, preds, name="EffNet")
    
    global_step = tf.Variable(0, trainable=False)
    initial_learning_rate = 0.01
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=1000,#100000
        decay_rate=0.96,
        staircase=True)

    opt = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
    #opt = tfa.optimizers.RectifiedAdam(learning_rate=0.001)
    #opt = tfa.optimizers.SGDW(learning_rate=0.001,weight_decay=0.001, momentum=0.01) #weight_decay=0.001
    #opt = tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001), 100, 10)
    #loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
    #opt=tfa.optimizers.SWA(tf.keras.optimizers.Adam(lr=0.001))
    #model.compile(loss=mloss(0.5), optimizer="adam", metrics=[kloss])
    model.compile(loss=mloss(0.5), optimizer=opt, metrics=[kloss])
    #model.compile(optimizer=opt,loss=loss,metrics=['AUC'])

    return model


In [ ]:
net = InceptionV3_GoogleNet()

In [ ]:
net.summary()

In [ ]:
# net = EffNet()
# print(net.summary())

In [ ]:
x_min = np.min(x)
x_max = np.max(x)
xs = x - x_min / (x_max - x_min)

In [ ]:
xs.shape, y.shape, x_min

In [ ]:
#Redifining xs in order to agregate a one dimenssion
#from array (1231, 128, 128) to (1231, 128, 128, 1)
xs = np.expand_dims(xs, axis=3)
xs.shape

In [ ]:
#Splitting data in train and developepment set
from sklearn.model_selection import train_test_split
X_train, X_oof, y_train, y_oof = train_test_split(xs, y, test_size=0.1, random_state=42)

In [ ]:
y

In [ ]:
#Defining to save the best model
def checkpoint_callback(counter):
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='./fold'+str(counter)+'.h5',
        save_weights_only=True,
        monitor='val_kloss',
        mode='min',
        save_best_only=True)
    
    return model_checkpoint_callback

In [ ]:
def display_graph(history):
    plt.plot(history.history['kloss'])
    plt.plot(history.history['val_kloss'])
    plt.legend(['train', 'valid.'])
    plt.grid()
    plt.title("Curves Comparision")
    #finding and plotting the min val_kloss value
    min_val_kloss=np.min(history.history['val_kloss'])
    x_min_val_kaloss=np.where(history.history['val_kloss']==np.min(history.history['val_kloss']))[0][0]
    plt.scatter(x_min_val_kaloss,min_val_kloss, c='#ff7f0e', s=200)
    bbox = dict(boxstyle="round", fc="0.8")
    plt.annotate("min_val_kloss: "+str(np.round(min_val_kloss, decimals=2)), (x_min_val_kaloss,min_val_kloss+0.35),bbox=bbox)
    plt.annotate("Iteration: "+str(x_min_val_kaloss+1), (x_min_val_kaloss,min_val_kloss+0.7),bbox=bbox)
    plt.show()

In [ ]:
SEED=42
FOLDS=5

skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)

#skf.get_n_splits(xs)
skf

# Trainning models using Kfolds

In [ ]:
def kloss1(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float64)
    tf.dtypes.cast(y_pred, tf.float64)
    sigma = y_pred[:, 1]
    fvc_pred = y_pred[:, 0]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, tf.dtypes.cast(C1, dtype="float64"))
    delta = tf.abs(y_true - fvc_pred)
    delta = tf.minimum(tf.dtypes.cast(delta, dtype="float64"), tf.dtypes.cast(C2, dtype="float64"))
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float64) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [ ]:
counter=1
EPOCHS=65
for train , test in skf.split(X_train):
	#print('train: %s, test: %s' % (train,test))
    
    net = InceptionV3_GoogleNet()

    print()
    print('Fold '+str(counter)+':')

    history=net.fit(X_train[train], y_train[train],
                    batch_size=32, epochs=EPOCHS,
                    validation_data=(X_train[test], y_train[test]),
                    callbacks=[checkpoint_callback(counter)])
    print('kloss out of fold: '+str(kloss1(y_oof, net.predict(X_oof, batch_size=100, verbose=1)).numpy()))
    display_graph(history)
    
    print()
    print()

    counter+=1

In [ ]:
#history=net.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test), callbacks=[model_checkpoint_callback])

In [ ]:
#net.fit([xs, z], y, batch_size=32, epochs=120) #, validation_split=0.1

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
#net.load_weights('./weights.h5')

In [ ]:
#pred = net.predict([xs, z], batch_size=100, verbose=1)

In [ ]:
net1 = InceptionV3_GoogleNet()
net2 = InceptionV3_GoogleNet()
net3 = InceptionV3_GoogleNet()
net4 = InceptionV3_GoogleNet()
net5 = InceptionV3_GoogleNet()

In [ ]:
net1.load_weights('./fold1.h5')
net2.load_weights('./fold2.h5')
net3.load_weights('./fold3.h5')
net4.load_weights('./fold4.h5')
net5.load_weights('./fold5.h5')

In [ ]:
pred1 = net1.predict(xs, batch_size=100, verbose=1)
pred2 = net2.predict(xs, batch_size=100, verbose=1)
pred3 = net3.predict(xs, batch_size=100, verbose=1)
pred4 = net4.predict(xs, batch_size=100, verbose=1)
pred5 = net5.predict(xs, batch_size=100, verbose=1)

In [ ]:
pred=np.mean(np.array([pred1,pred2,pred3,pred4,pred5]),axis=0)

In [ ]:
sigma_opt = mean_absolute_error(y, pred[:, 0])
sigma_mean = np.mean(pred[:, 1])
print(sigma_opt, sigma_mean)

In [ ]:
plt.plot(y)
plt.plot(pred[:, 0])
#plt.plot(pred[:, 1])

In [ ]:
pred[:, 1].min(), pred[:, 1].max()

In [ ]:
plt.hist(pred[:, 1])
plt.title("uncertainty in prediction")
plt.show()

### PREDICTION

In [ ]:
xe, df_te = get_images(sub, how="test")
df_te = df_te.merge(sub, how="left", on=['Patient', 'Weeks'])

In [ ]:
x_te = xe - x_min / (x_max - x_min)
#Adding new dimenssion
x_te = np.expand_dims(x_te, axis=3)
ze = df_te[FE].values

In [ ]:
pe1 = net1.predict(xs, batch_size=100, verbose=1)
pe2 = net2.predict(xs, batch_size=100, verbose=1)
pe3 = net3.predict(xs, batch_size=100, verbose=1)
pe4 = net4.predict(xs, batch_size=100, verbose=1)
pe5 = net5.predict(xs, batch_size=100, verbose=1)

In [ ]:
pe=np.mean(np.array([pe1,pe2,pe3,pe4,pe5]),axis=0)

In [ ]:
#pe = net.predict([x_te, ze], batch_size=100, verbose=1)
pe = net.predict(x_te, batch_size=100, verbose=1)

In [ ]:
df_te['FVC1'] = pe[:, 0]
df_te['Confidence1'] = pe[:, 1]

In [ ]:
sub = sub.merge(df_te[['Patient','Weeks','FVC1','Confidence1']], how='left', 
                on=['Patient', 'Weeks'])
#====================================================#

In [ ]:
sub.head()

In [ ]:
subm = sub[['Patient_Week','FVC','Confidence','FVC1','Confidence1']].copy()

In [ ]:
subm.loc[~subm.FVC1.isnull()].head(10)

In [ ]:
subm.loc[~subm.FVC1.isnull(),'FVC'] = subm.loc[~subm.FVC1.isnull(),'FVC1']
if sigma_mean<70:
    subm['Confidence'] = sigma_opt
else:
    subm.loc[~subm.FVC1.isnull(),'Confidence'] = subm.loc[~subm.FVC1.isnull(),'Confidence1']
    subm.loc[subm.FVC1.isnull(),'Confidence'] = sigma_opt
#

In [ ]:
subm.head()

In [ ]:
subm.describe().T

In [ ]:
subm[["Patient_Week","FVC","Confidence"]].to_csv("submission.csv", index=False)

In [ ]:
tf.version.VERSION

In [ ]:
keras.__version__

In [ ]:
# import os
# os.chdir(r'./')
# from IPython.display import FileLink
# FileLink(r'./weights.h5')

In [ ]:
print()